In [13]:
# 1. Facebook Prophet su papildomais regresoriais
# Kodėl pasirinkti?
# Paprastas nustatyti ir naudoti.
# Automatiškai modeliuoja sezoniškumą, įskaitant paros ritmą.
# Leidžia pridėti orų prognozės duomenis kaip papildomus regresorius.

# pip install prophet

from prophet import Prophet
import pandas as pd

# Duomenys
data = pd.DataFrame({
    'ds': pd.date_range(start='2023-01-01', periods=7, freq='h'),  # Laiko žymės
    'y': [50, 60, 55, 48, 50, 52, 65],  # Elektros suvartojimas
    'temperature': [5, 6, 5, 4, 3, 2, 3],  # Temperatūra
    'humidity': [80, 82, 78, 76, 75, 74, 73]  # Drėgmė
})

# Sukurkite Prophet modelį
model = Prophet(daily_seasonality=True)  # Paros sezoniškumas
model.add_regressor('temperature')
model.add_regressor('humidity')

# Treniruotė
model.fit(data)

# Prognozavimas
future = model.make_future_dataframe(periods=17, freq='h')  # 24 valandų prognozė
future['temperature'] = [4] * 24  # Pavyzdinė temperatūra
future['humidity'] = [70] * 24  # Pavyzdinė drėgmė
forecast = model.predict(future)

# Rezultatai
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

21:21:55 - cmdstanpy - INFO - Chain [1] start processing
21:21:56 - cmdstanpy - INFO - Chain [1] done processing


                    ds        yhat  yhat_lower  yhat_upper
0  2023-01-01 00:00:00  122.129941  122.129941  122.129941
1  2023-01-01 01:00:00  126.339239  126.339239  126.339239
2  2023-01-01 02:00:00  107.649749  107.649749  107.649749
3  2023-01-01 03:00:00  106.440401  106.440401  106.440401
4  2023-01-01 04:00:00  123.971129  123.971129  123.971129
5  2023-01-01 05:00:00  141.501867  141.501867  141.501867
6  2023-01-01 06:00:00  119.490974  119.490974  119.490974
7  2023-01-01 07:00:00   70.940745   68.491316   73.227001
8  2023-01-01 08:00:00   38.436504   30.672188   45.468573
9  2023-01-01 09:00:00   44.001073   29.433163   58.192296
10 2023-01-01 10:00:00   55.601966   31.419854   79.588449
11 2023-01-01 11:00:00   12.810398  -23.313333   45.587369
12 2023-01-01 12:00:00 -107.030646 -154.725107  -61.297299
13 2023-01-01 13:00:00 -248.149424 -308.231384 -187.327539
14 2023-01-01 14:00:00 -309.490368 -383.002068 -234.775639
15 2023-01-01 15:00:00 -230.131066 -316.722255 -141.0729

In [2]:
# 2. SARIMAX (ARIMA su eksogeniniais kintamaisiais)
# Kodėl pasirinkti?
# Gerai tinka modeliuoti laiko priklausomybes (pvz., paros sezoniškumą).
# Leidžia įtraukti orų prognozes kaip papildomus veiksnius.

# pip install statsmodels

import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Duomenys
data = pd.DataFrame({
    'consumption': [50, 60, 55, 48, 50, 52, 65],  # Elektros suvartojimas
    'temperature': [5, 6, 5, 4, 3, 2, 3],  # Temperatūra
    'humidity': [80, 82, 78, 76, 75, 74, 73]  # Drėgmė
})

# Tikslinis ir eksogeniniai kintamieji
y = data['consumption']
X = data[['temperature', 'humidity']]

# SARIMAX modelis su sezoniniu paros efektu
model = SARIMAX(y, exog=X, order=(1, 1, 1), seasonal_order=(0, 1, 1, 24))
results = model.fit()

# Prognozė su naujais orų duomenimis
future_X = pd.DataFrame({'temperature': [4] * 24, 'humidity': [70] * 24})
forecast = results.get_forecast(steps=24, exog=future_X)
print(forecast.summary_frame())

consumption        mean      mean_se  mean_ci_lower  mean_ci_upper
7             65.000000  1000.000500   -1894.964965    2024.964965
8             65.000000  1414.214269   -2706.809035    2836.809035
9             65.000000  1732.051674   -3329.758900    3459.758900
10            65.000000  2000.001000   -3854.929929    3984.929929
11            65.000000  2236.069096   -4317.614894    4447.614894
12            65.000000  2449.490968   -4735.914077    4865.914077
13            65.000000  2645.752634   -5120.579875    5250.579875
14            65.000000  2828.428539   -5478.618069    5608.618069
15            65.000000  3000.001500   -5814.894894    5944.894894
16            65.000000  3162.279241   -6132.953422    6262.953422
17            65.000000  3316.626449   -6435.468390    6565.468390
18            65.000000  3464.103347   -6724.517799    6854.517799
19            65.000000  3605.553078   -7001.754178    7131.754178
20            65.000000  3741.659258   -7268.517387    7398.51

C:\Users\Eridas\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\Eridas\AppData\Roaming\Python\Python312\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [17]:
# 3. Linijinė regresija su kategoriniais kintamaisiais
# Kodėl pasirinkti?
# Paprasta implementuoti ir interpretuoti.
# Paros laiko efektą galima įtraukti kaip kategorinį kintamąjį (pvz., valandos).

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# Duomenys
data = pd.DataFrame({
    'hour': [0, 1, 2, 3, 4, 5, 6],  # Valanda
    'temperature': [5, 6, 5, 4, 3, 2, 3],  # Temperatūra
    'humidity': [80, 82, 78, 76, 75, 74, 73],  # Drėgmė
    'consumption': [50, 60, 55, 48, 50, 52, 65]  # Elektros suvartojimas
})

# Kategoriniai kintamieji (valanda)
encoder = OneHotEncoder(sparse_output=False) # 'sparse_output'
hour_encoded = encoder.fit_transform(data[['hour']])

# Modelio treniruotė
X = pd.concat([pd.DataFrame(hour_encoded), data[['temperature', 'humidity']]], axis=1)
X.columns = X.columns.astype(str)  # Užtikriname, kad pavadinimai būtų string tipo
y = data['consumption']
model = LinearRegression()
model.fit(X, y)

# Prognozė naujai dienai
future_hours = encoder.transform([[0], [1], [2], [3], [4], [5], [6]])
future_weather = pd.DataFrame({'temperature': [4] * 7, 'humidity': [70] * 7})
future_X = pd.concat([pd.DataFrame(future_hours), future_weather], axis=1)
future_X.columns = future_X.columns.astype(str)  # Užtikriname, kad pavadinimai būtų string tipo
forecast = model.predict(future_X)

print("Prognozės:", forecast)


Prognozės: [59.18794607 68.72640761 61.77557494 55.2371134  58.90483743 62.57256146
 71.4924663 ]


C:\Users\Eridas\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [8]:
# 3. LSTM (giluminio mokymosi metodas)
# Kada naudoti: Kai duomenys turi sudėtingas nelinearines priklausomybes arba ilgesnę laiko priklausomybę (pvz., savaitinį ar mėnesinį ciklą).
# Įrankis: TensorFlow arba PyTorch

# pip install tensorflow

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Duomenys
X = np.array([
    [[20, 60], [21, 65], [19, 63]],  # Ankstesni meteorologiniai duomenys
    [[21, 65], [19, 63], [18, 59]]
])  # Formatas: (pavyzdžiai, laiko momentai, savybės)
y = np.array([120, 130])  # Tiksliniai duomenys

# Modelio kūrimas
model = Sequential([
    LSTM(50, activation='relu', input_shape=(3, 2)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Treniruotė
model.fit(X, y, epochs=100, verbose=0)

# Prognozė
X_new = np.array([[[19, 63], [18, 59], [22, 61]]])
print("Prognozė:", model.predict(X_new))

ModuleNotFoundError: No module named 'tensorflow'